In [21]:
from sklearn import svm 
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
%matplotlib inline

#clusters
import scipy 
import scipy.stats as stats
import matplotlib.pyplot as plt
from IPython.display import Image
from scipy import misc
from scipy.spatial import distance
from scipy.cluster import hierarchy
from sklearn import cluster
from sklearn import svm
from sklearn import datasets
from sklearn import mixture
from sklearn.model_selection import cross_val_score
from mpl_toolkits.mplot3d import axes3d
from ipywidgets import interact
from matplotlib import cm
from matplotlib.colors import LogNorm
np.set_printoptions(precision=1)

#tree
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from IPython.display import Image # displaying images files in jupyter
from IPython.display import IFrame # displaying pdf file in jupyter

import warnings
warnings.simplefilter("ignore")

In [2]:
# Create a SQL connection to our SQLite database
con = sqlite3.connect("soccerdata.sqlite")

match =  pd.read_sql_query("SELECT * FROM Match", con)
team =  pd.read_sql_query("SELECT * FROM Team", con)

#Be sure to close the connection.
con.close()

In [3]:
#match data
match = match.loc[match['league_id'] == 1729,] #filter by league
match = match.iloc[:,3:11] #only need first 11 columns
match.index = range(1, len(match["stage"]) + 1)
match["Outcome"] = 0

#rename columns
match.columns = ["Season", "Stage", "Date", "Match_ID", "Home_Team_ID", "Away_Team_ID", "Home_Goals", "Away_Goals", "Outcome"]

In [4]:
#team data
team = team.drop("team_fifa_api_id", 1)

#extract those teams that took part in league = 1729
hometeams = match["Home_Team_ID"].unique()
awayteams = match["Away_Team_ID"].unique()
leagueTeams = np.concatenate((hometeams, awayteams), axis = 0)
team = team.loc[team['team_api_id'].isin(leagueTeams) ,] 

#print team
#print(team)

      id  team_api_id           team_long_name team_short_name
25  3457        10260        Manchester United             MUN
26  3458        10261         Newcastle United             NEW
27  3459         9825                  Arsenal             ARS
28  3460         8659     West Bromwich Albion             WBA
29  3461         8472               Sunderland             SUN
30  3462         8650                Liverpool             LIV
31  3463         8654          West Ham United             WHU
32  3464         8528           Wigan Athletic             WIG
33  3465        10252              Aston Villa             AVL
34  3466         8456          Manchester City             MCI
35  3467         8668                  Everton             EVE
36  3468         8655         Blackburn Rovers             BLB
37  3469         8549            Middlesbrough             MID
38  3470         8586        Tottenham Hotspur             TOT
39  3471         8559         Bolton Wanderers         

In [5]:
#create a class variable for "Win", "Lose", "Tie" - WRT home team
for i in range(1, len(match["Outcome"]) + 1):
    home_goals = match.loc[i, "Home_Goals"]
    away_goals = match.loc[i, "Away_Goals"]
    if home_goals > away_goals:
        match.loc[i,'Outcome'] = "Win"
    elif home_goals == away_goals:
        match.loc[i,'Outcome'] = "Tie"
    elif home_goals < away_goals:
        match.loc[i,'Outcome'] = "Lose"

In [ ]:
match.head()

In [6]:
def getLastXMatchesResults_Home(date, team, x = 10):    
    #Filter team matches from matches
    teamMatches = match[(match['Home_Team_ID'] == team)]                           
    #Filter x last matches from team matches
    xMatches = teamMatches[teamMatches.Date < date].sort_values(by = 'Date', ascending = False).iloc[0:x,:]
    #Calculate win percentage
    won = len(xMatches['Outcome'] == "Win") / 10
    return won

def getLastXMatchesResults_Away(date, team, x = 10):    
    #Filter team matches from matches
    teamMatches = match[(match['Away_Team_ID'] == team)]                           
    #Filter x last matches from team matches
    xMatches = teamMatches[teamMatches.Date < date].sort_values(by = 'Date', ascending = False).iloc[0:x,:]
    #Calculate win percentage
    won = len(xMatches['Outcome'] == "Win") / 10
    return won
    
def getLastXMatchesResultsAgainstEachOther(date, home_team, away_team, x = 10):    
    #Find matches of both teams
    homeMatches = match[(match['Home_Team_ID'] == home_team) & (match['Away_Team_ID'] == away_team)]    
    awayMatches = match[(match['Home_Team_ID'] == away_team) & (match['Away_Team_ID'] == home_team)] 
    #Get last x matches - Home
    try:    
        xMatches = homeMatches[homeMatches.Date < date].sort_values(by = 'Date', ascending = False).iloc[0:x,:]
        homeWin = len(xMatches['Outcome'] == "Win") / 10
    except:
        xMatches = homeMatches[homeMatches.Date < date].sort_values(by = 'Date', ascending = False).iloc[0:homeMatches.shape[0],:]
        homeWin = 0
        #Check for error in data
        if(xMatches.shape[0] > x):
            print("Error in obtaining matches")
    
    #Get last x matches - Away
    try:    
        xMatches = awayMatches[awayMatches.Date < date].sort_values(by = 'Date', ascending = False).iloc[0:x,:]
        awayWin = len(xMatches['Outcome'] == "Win") / 10
    except:
        xMatches = awayMatches[awayMatches.Date < date].sort_values(by = 'Date', ascending = False).iloc[0:awayMatches.shape[0],:]
        awayWin = 0
        #Check for error in data
        if(xMatches.shape[0] > x):
            print("Error in obtaining matches")
    
            
    #Return data
    return np.array([homeWin, awayWin])


In [15]:
#create columns for last X matches
match["HomeTeamWins_Home"] = 0
match["HomeTeamWins_Away"] = 0
match["AwayTeamWins_Home"] = 0
match["AwayTeamWins_Away"] = 0
match["HomeVsAway_Home"] = 0
match["HomeVsAway_Away"] = 0

for i in range(1, len(match["Match_ID"]) + 1):
    match.loc[i, "HomeTeamWins_Home"] = getLastXMatchesResults_Home(match.loc[i, "Date"],  match.loc[i, "Home_Team_ID"])
    match.loc[i, "HomeTeamWins_Away"] = getLastXMatchesResults_Away(match.loc[i, "Date"],  match.loc[i, "Home_Team_ID"])
    match.loc[i, "AwayTeamWins_Home"] = getLastXMatchesResults_Home(match.loc[i, "Date"],  match.loc[i, "Away_Team_ID"])
    match.loc[i, "AwayTeamWins_Away"] = getLastXMatchesResults_Away(match.loc[i, "Date"],  match.loc[i, "Away_Team_ID"])
    temp =  getLastXMatchesResultsAgainstEachOther(match.loc[i, "Date"],match.loc[i, "Home_Team_ID"],match.loc[i, "Away_Team_ID"])
    match.loc[i, "HomeVsAway_Home"] = temp[0]
    match.loc[i, "HomeVsAway_Away"] = temp[1]

In [ ]:
#use clusters to classify
X = match[["Home_Team_ID", "Away_Team_ID", "HomeTeamWins_Home", "HomeTeamWins_Away", "AwayTeamWins_Home", "AwayTeamWins_Away", "HomeVsAway_Home", "HomeVsAway_Away"]]
Y =  match['Outcome']
kmeans = cluster.KMeans(n_clusters = 3).fit(X)

#predict - find the cluster of each observation
clusters = kmeans.predict(X)

In [ ]:
print(clusters[:20])
print(match.loc[0:20,'Outcome'])

In [ ]:
#map the clusters to 'Win', 'Lose' or 'Tie'
cluster_mapping = dict(zip([2,1,0],['Win','Lose', 'Tie']))
cluster_mapping

In [ ]:
# use clustering to infer the label of each observation
classes = list(map(lambda k:cluster_mapping[k], kmeans.labels_))
print('True label:', Y[:20].tolist())
print('Prediction:', classes[:20])

In [ ]:
(Y==classes).sum()/Y.shape[0]

In [55]:
#create training set and validation set
#season 2008-09 to 2014-15 will be the training data
#we will predict the outcome of the matches in 2015-16
train = match.loc[match["Season"] != "2015/2016",]
validation = match.loc[match["Season"] == "2015/2016",]

#trees
X = train[["Home_Team_ID", "Away_Team_ID", "HomeTeamWins_Home", "HomeTeamWins_Away", "AwayTeamWins_Home", "AwayTeamWins_Away", "HomeVsAway_Home", "HomeVsAway_Away"]]
Y =  train['Outcome']
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X,Y)

In [56]:
# predict values for training data
train['prediction'] = clf.predict(X)

#tree depth
print(clf.tree_.max_depth)

29


In [57]:
#try an out of sample prediction
X = validation[["Home_Team_ID", "Away_Team_ID", "HomeTeamWins_Home", "HomeTeamWins_Away", "AwayTeamWins_Home", "AwayTeamWins_Away", "HomeVsAway_Home", "HomeVsAway_Away"]]
Y = validation['Outcome']
validation['prediction'] = clf.predict(X)

#prediction errors
print((validation['prediction'] != validation['Outcome']).sum())

#accuracy
print((validation['prediction'] != validation['Outcome']).sum() / len(validation['Outcome']))

241
0.634210526316


In [68]:
#random forest on train data
X = train[["Home_Team_ID", "Away_Team_ID", "HomeTeamWins_Home", "HomeTeamWins_Away", "AwayTeamWins_Home", "AwayTeamWins_Away", "HomeVsAway_Home", "HomeVsAway_Away"]]
Y =  train['Outcome']
clf_rf = RandomForestClassifier(max_features="sqrt")
clf_rf = clf_rf.fit(X, Y)
clf_rf_scores = cross_val_score(clf_rf, X, Y, cv=5)

In [69]:
# predict values for training data
train['prediction'] = clf_rf.predict(X)

In [66]:
#try an out of sample prediction
X = validation[["Home_Team_ID", "Away_Team_ID", "HomeTeamWins_Home", "HomeTeamWins_Away", "AwayTeamWins_Home", "AwayTeamWins_Away", "HomeVsAway_Home", "HomeVsAway_Away"]]
Y = validation['Outcome']
validation['prediction'] = clf_rf.predict(X)

#prediction errors
print((validation['prediction'] != validation['Outcome']).sum())

#accuracy
print((validation['prediction'] != validation['Outcome']).sum() / len(validation['Outcome']))

237
0.623684210526


In [83]:
#try boosting
X = train[["Home_Team_ID", "Away_Team_ID", "HomeTeamWins_Home", "HomeTeamWins_Away", "AwayTeamWins_Home", "AwayTeamWins_Away", "HomeVsAway_Home", "HomeVsAway_Away"]]
Y =  train['Outcome']
clf = AdaBoostClassifier()
clf = clf.fit(X,Y)
clf.score(X,Y)

0.52518796992481198

In [84]:
#try an out of sample prediction
X = validation[["Home_Team_ID", "Away_Team_ID", "HomeTeamWins_Home", "HomeTeamWins_Away", "AwayTeamWins_Home", "AwayTeamWins_Away", "HomeVsAway_Home", "HomeVsAway_Away"]]
Y = validation['Outcome']
validation['prediction'] = clf.predict(X)

#prediction errors
print((validation['prediction'] != validation['Outcome']).sum())

#accuracy
print((validation['prediction'] != validation['Outcome']).sum() / len(validation['Outcome']))

223
0.586842105263
